# Simple example

In [1]:
# Add BipartitePandas to system path, do not run this
# import sys
# sys.path.append('../../..')

## Import the BipartitePandas Package

Make sure to install it using `pip install bipartitepandas`

In [2]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

In this example, we simulate data (we set parameters to make data cleaning interesting)

In [3]:
df = bpd.SimBipartite(bpd.sim_params({'firm_size': 10, 'p_move': 0.05})).simulate()
display(df)

,i,j,y,t,l,k,alpha,psi
0,0,478,-0.354375,0,0,4,-0.967422,-0.114185
1,0,478,-1.370253,1,0,4,-0.967422,-0.114185
2,0,478,-1.342482,2,0,4,-0.967422,-0.114185
3,0,478,-0.046391,3,0,4,-0.967422,-0.114185
4,0,478,-2.302898,4,0,4,-0.967422,-0.114185
...,...,...,...,...,...,...,...,...
49995,9999,567,1.558651,0,3,5,0.430727,0.114185
49996,9999,567,-0.257804,1,3,5,0.430727,0.114185
49997,9999,567,0.494844,2,3,5,0.430727,0.114185
49998,9999,567,-1.472770,3,3,5,0.430727,0.114185


## Columns

BipartitePandas includes seven pre-defined general columns:

#### Required
- $i$: worker id (any type)
- $j$: firm id (any type)
- $y$: income (float or int)

#### Optional
- $t$: time (int)
- $g$: firm type (any type)
- $w$: weight (float or int)
- $m$: move indicator (int)

## Formats

BipartitePandas includes four formats:

- *Long* - each row gives a single observation
- *Collapsed Long* - like *Long*, but employment spells at the same firm are collapsed into a single observation
- *Event Study* - each row gives two consecutive observations
- *Collapsed Event Study* - like *Event Study*, but employment spells at the same firm are collapsed into a single observation

These formats divide general columns differently:

- *Long* - $i$, $j$, $y$, $t$, $g$, $w$, $m$
- *Collapsed Long* - $i$, $j$, $y$, $t1$, $t2$, $g$, $w$, $m$
- *Event Study* - $i$, $j1$, $j2$, $y1$, $y2$, $t1$, $t2$, $g1$, $g2$, $w1$, $w2$, $m$
- *Collapsed Event Study* - $i$, $j1$, $j2$, $y1$, $y2$, $t11$, $t12$, $t21$, $t22$, $g1$, $g2$, $w1$, $w2$, $m$

## Constructing DataFrames

Our simulated data is in *Long* format. How do we construct a *Long* dataframe?

In [4]:
i = df['i']
j = df['j']
y = df['y']
t = df['t']
bdf = bpd.BipartiteDataFrame(i=i, j=j, y=y, t=t)
display(bdf)

,i,j,y,t
0,0,478,-0.354375,0
1,0,478,-1.370253,1
2,0,478,-1.342482,2
3,0,478,-0.046391,3
4,0,478,-2.302898,4
...,...,...,...,...
49995,9999,567,1.558651,0
49996,9999,567,-0.257804,1
49997,9999,567,0.494844,2
49998,9999,567,-1.472770,3


Are we sure this is long? Let's check the datatype:

In [5]:
type(bdf)

bipartitepandas.bipartitelong.BipartiteLong

This method works to construct any format! Just make sure not to mix up columns between formats.

## Now that we have our dataframe, let's check out some statistics

In [6]:
bdf.summary()

format: 'BipartiteLong'
number of workers: 10000
number of firms: 999
number of observations: 50000
mean wage: 0.028111816901488847
median wage: 0.04090439077720731
min wage: -5.792275639518705
max wage: 6.780836917708111
var(wage): 2.7016060571108937
no NaN values: False
no duplicates: False
i-t (worker-year) observations unique (None if t column(s) not included): False
no returns (None if not yet computed): None
contiguous 'i' ids (None if not included): False
contiguous 'j' ids (None if not included): False
contiguous 'g' ids (None if not included): None
connectedness (None if ignoring connectedness): None


## Now let's clean our data - and make sure the result is leave-one-observation-out connected

*Hint:* want details on all cleaning parameters? Type `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then type `bpd.clean_params().describe(key)`.

In [7]:
bdf = bdf.clean(bpd.clean_params({'connectedness': 'leave_out_observation'}))
display(bdf)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how='leave_out_observation')
making 'i' ids contiguous
making 'j' ids contiguous
sorting columns
resetting index


,i,j,y,t,m
0,0,0,-0.354375,0,0
1,0,0,-1.370253,1,0
2,0,0,-1.342482,2,0
3,0,0,-0.046391,3,0
4,0,0,-2.302898,4,0
...,...,...,...,...,...
44593,8960,313,1.558651,0,0
44594,8960,313,-0.257804,1,0
44595,8960,313,0.494844,2,0
44596,8960,313,-1.472770,3,0


We can check how the summary statistics changed:

In [8]:
bdf.summary()

format: 'BipartiteLong'
number of workers: 8961
number of firms: 874
number of observations: 44598
mean wage: 0.02037605247576551
median wage: 0.03303561912360109
min wage: -5.792275639518705
max wage: 6.780836917708111
var(wage): 2.7305577342520446
no NaN values: True
no duplicates: True
i-t (worker-year) observations unique (None if t column(s) not included): True
no returns (None if not yet computed): True
contiguous 'i' ids (None if not included): True
contiguous 'j' ids (None if not included): True
contiguous 'g' ids (None if not included): None
connectedness (None if ignoring connectedness): 'leave_out_observation'


## Converting formats

### *Collapsed* format

In [9]:
display(bdf.collapse())

,i,j,y,t1,t2,w,m
0,0,0,-1.083280,0,4,5,0
1,1,1,-2.073813,0,4,5,0
2,2,2,-0.696272,0,4,5,0
3,3,3,1.237246,0,0,1,1
4,3,4,0.792275,1,4,4,1
...,...,...,...,...,...,...,...
10873,8957,873,0.283228,0,4,5,0
10874,8958,593,2.411665,0,1,2,1
10875,8958,354,2.321508,2,4,3,1
10876,8959,518,0.777356,0,4,5,0


### *Event Study* format

In [10]:
display(bdf.to_eventstudy())

,i,j1,j2,y1,y2,t1,t2,m
0,0,0,0,-0.354375,-0.354375,0,0,0
1,0,0,0,-1.370253,-1.370253,1,1,0
2,0,0,0,-1.342482,-1.342482,2,2,0
3,0,0,0,-0.046391,-0.046391,3,3,0
4,0,0,0,-2.302898,-2.302898,4,4,0
...,...,...,...,...,...,...,...,...
42800,8960,313,313,1.558651,1.558651,0,0,0
42801,8960,313,313,-0.257804,-0.257804,1,1,0
42802,8960,313,313,0.494844,0.494844,2,2,0
42803,8960,313,313,-1.472770,-1.472770,3,3,0


### *Collapsed Event Study* format

In [11]:
display(bdf.collapse().to_eventstudy())

,i,j1,j2,y1,y2,t11,t12,t21,t22,w1,w2,m
0,0,0,0,-1.083280,-1.083280,0,4,0,4,5,5,0
1,1,1,1,-2.073813,-2.073813,0,4,0,4,5,5,0
2,2,2,2,-0.696272,-0.696272,0,4,0,4,5,5,0
3,3,3,4,1.237246,0.792275,0,0,1,4,1,4,1
4,4,5,5,0.598853,0.598853,0,4,0,4,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9080,8956,871,871,-1.639261,-1.639261,0,4,0,4,5,5,0
9081,8957,873,873,0.283228,0.283228,0,4,0,4,5,5,0
9082,8958,593,354,2.411665,2.321508,0,1,2,4,2,3,1
9083,8959,518,518,0.777356,0.777356,0,4,0,4,5,5,0


## Generating firm clusters

Notice the new $g$ column

In [12]:
display(bdf.cluster())

INFO:bipartitelong:firm groups computed
INFO:bipartitelong:dictionary linking firms to clusters generated
INFO:bipartitelong:sorting columns
INFO:bipartitelong:clusters merged into data


,i,j,y,t,g,m
0,0,0,-0.354375,0,7,0
1,0,0,-1.370253,1,7,0
2,0,0,-1.342482,2,7,0
3,0,0,-0.046391,3,7,0
4,0,0,-2.302898,4,7,0
...,...,...,...,...,...,...
44593,8960,313,1.558651,0,4,0
44594,8960,313,-0.257804,1,4,0
44595,8960,313,0.494844,2,4,0
44596,8960,313,-1.472770,3,4,0
